## Load packages

In [ ]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
import yaml
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions


## Load Configs

In [ ]:
role = get_execution_role()
region = boto3.Session().region_name

feature_group_name = "taxi-feature-group-no-encoding"
bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/xgboost_model"

In [ ]:
bucket

## Get data from Feature Store

In [ ]:
# Connect to the Feature Store
boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=feature_store_session)

In [ ]:
# Build SQL query to features group
fs_query = feature_group.athena_query()
fs_table = fs_query.table_name
query_string = 'SELECT * FROM "'+fs_table+'"'
print('Running ' + query_string)

In [ ]:
# Run Athena query. The output is loaded to a Pandas dataframe.
fs_query.run(query_string=query_string, output_location='s3://'+bucket+'/'+prefix+'/fs_query_results/')
fs_query.wait()
model_data = fs_query.as_dataframe()

In [ ]:
model_data = model_data.drop(['fs_id', 'fs_time', 'write_time', 'api_invocation_time', 'is_deleted'], axis=1)

In [ ]:
model_data.head()

In [ ]:
model_data.shape

## Prepare Dataset for Training, Test, and Validation

In [ ]:
# Randomly sort the data then split out first 70%, second 20%, and last 10%
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])

In [ ]:
# Move the target variable to the first column
pd.concat([train_data['trip_total'], train_data.drop(['trip_total'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['trip_total'], validation_data.drop(['trip_total'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

In [ ]:
# Upload the training and validation dataset into S3 bucket.
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

## Training

In [ ]:
# Select the xgboost built in container
container = sagemaker.image_uris.retrieve(region=region, framework='xgboost', version='1.7-1')

# Get the input and validation path
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

sess = sagemaker.Session()

hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "num_round":"500"}

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess,
                                    hyperparameters=hyperparameters
                                   )

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

## Host the model

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Cleanup

In [ ]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)